Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [359]:
from dataclasses import asdict

import fhirclient
import requests
from fhirclient.models.fhirinstant import FHIRInstant
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
#server = "https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/"
server = os.getenv("FHIR_SERVER")

api_url = server + "metadata"
print(api_url)
response = requests.get(api_url)
#response.json()

http://192.168.1.67/healthconnect/cdr/fhir/r4/metadata


Find a patient. Simple search for a patient named wrexham.

The data used here is a combination of patient demographics from EHR systems (supplied with the orders) and [NHS England Personal Demographics Service - FHIR API](https://digital.nhs.uk/developer/api-catalogue/personal-demographics-service-fhir)

In [360]:


api_url = server + "Patient?name=wrexham"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
patientJSON = response.json()
print(patientJSON)

patientId = None


if ((patientJSON['total']> 0) and (len(patientJSON['entry'])>0)):
    patientId = patientJSON['entry'][0]['resource']['id']
    print()
    print("Patient = " + patientId)

http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham
{'resourceType': 'Bundle', 'id': 'd8502232-109b-4bd2-8316-7d7fc569f6bb', 'type': 'searchset', 'timestamp': '2025-12-30T09:01:53Z', 'total': 1, 'link': [{'relation': 'self', 'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham'}], 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient/18', 'resource': {'resourceType': 'Patient', 'address': [{'id': 'qEFQG', 'line': ['30 PASTURE ROAD', 'BARTON-UPON-HUMBER', 'S HUMBERSIDE'], 'period': {'start': '2025-06-24'}, 'postalCode': 'DN18 5HU', 'use': 'home'}], 'birthDate': '1987-02-18', 'gender': 'unknown', 'generalPractitioner': [{'id': 'ZGilw', 'identifier': {'period': {'start': '2025-03-02'}, 'system': 'https://fhir.nhs.uk/Id/ods-organization-code', 'value': 'A20047'}, 'reference': 'Organization/1', 'type': 'Organization'}], 'id': '18', 'identifier': [{'system': 'https://fhir.nhs.uk/Id/nhs-number', 'type': {'coding': [{'code': 'NH', 'sy

Now lets find diagnostic reports for this patient.

In [361]:
organisationId = None

if patientId != None:
    api_url = server + "DiagnosticReport?patient="+"Patient/"+patientId
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    diagnosticReportsJSON = response.json()
    print(diagnosticReportsJSON)
    if (len(diagnosticReportsJSON['entry'])>0):
        organisationId = diagnosticReportsJSON['entry'][0]['resource']['performer'][0]['reference'].replace('Organization/', '')
        print()
        print("Organization = " + organisationId)


{'resourceType': 'Bundle', 'id': 'a828ecad-7b05-4395-ab1d-a2449882f613', 'type': 'searchset', 'timestamp': '2025-12-30T09:01:53Z', 'total': 1, 'link': [{'relation': 'self', 'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?patient=Patient%2F18'}], 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport/22', 'resource': {'resourceType': 'DiagnosticReport', 'basedOn': [{'identifier': {'assigner': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code', 'value': 'RBS'}}, 'system': 'https://nw-gmsa.github.io/Id/PLAC', 'type': {'coding': [{'code': 'PLAC', 'system': 'http://terminology.hl7.org/CodeSystem/v2-0203'}]}, 'value': '1000152867'}, 'type': 'ServiceRequest'}], 'category': [{'coding': [{'code': 'GE', 'system': 'http://terminology.hl7.org/CodeSystem/v2-0074'}]}], 'code': {'coding': [{'code': 'R125_Cardiology', 'display': 'PANEL: R125.1, R125.2 - Thoracic aortic aneurysm and dissection v1.2', 'system': 'IGEAP'}]}, 'id':

Can also return details about the organisation that created this.

This data is sourced from [NHS England Organisation Data Terminology - FHIR API](https://digital.nhs.uk/developer/api-catalogue/organisation-data-terminology). Note: the model used here is simplified.

In [362]:
if organisationId != None:
    api_url = server + "Organization/"+organisationId
    print(api_url)
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    organisationJSON = response.json()
    print(organisationJSON)

http://192.168.1.67/healthconnect/cdr/fhir/r4/Organization/5
{'resourceType': 'Organization', 'active': True, 'id': '5', 'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code', 'value': '699X0'}], 'name': 'NORTH WEST GLH LED BY MANCHESTER UNIVERSITY NHS FOUNDATION TRUST', 'partOf': {'display': 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST', 'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code', 'value': 'R0A'}, 'reference': 'Organization/4'}, 'type': [{'coding': [{'code': 'RO173', 'display': 'PATHOLOGY LAB', 'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]}], 'meta': {'lastUpdated': '2025-12-19T11:20:46Z', 'versionId': '2'}}


Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [363]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response1JSON = response.json()

print(response1JSON['total'])

reports = []
for entry in response1JSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

8
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
R125.1
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-12-17T12:22:52+00:00
SANGER-Family-follow-up-Rare
R375.1
DiagnosticReport 2025-12-17T12:25:36+00:00
M80_AML_GX
M80.2
DiagnosticReport 2025-12-18T12:14:40+00:00
PCR-FLT3-ITD
M80.18
M80.21
DiagnosticReport 2025-12-06T13:37:35+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-08-14T09:41:16+00:00
R240.1
R240.1


We are aiming at display graphs based on the order and the report.

We can include the order which in FHIR is held in ServiceRequest by including this in the query.

In [364]:
import fhirclient.models.servicerequest as sr
import fhirclient.models.specimen as sp

serviceRequests = []
diagnosticReports = []
specimens = []

api_url = server + "DiagnosticReport?_include=DiagnosticReport:based-on&_include=DiagnosticReport:specimen"


while True:
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    responseInclude = response.json()

    #print(responseInclude)
    print(responseInclude['total'])
    entry = responseInclude['entry']
    print(len(entry))
    if len(entry) == 0:
        break
    for entry in responseInclude['entry']:

        if entry['resource']['resourceType'] == 'DiagnosticReport':
            report = dr.DiagnosticReport(entry['resource'])
            diagnosticReports.append(report)
        if entry['resource']['resourceType'] == 'ServiceRequest':
            request = sr.ServiceRequest(entry['resource'])
            serviceRequests.append(request)
        if entry['resource']['resourceType'] == 'Specimen':
            specimen = sp.Specimen(entry['resource'])
            specimens.append(specimen)

    print("ServiceRequest = " + str(len(serviceRequests)))
    print("DiagnosticReport = " + str(len(diagnosticReports)))
    print("Specimen = " + str(len(specimens)))

    found = False
    for link in responseInclude['link']:
        if link['relation'] == 'next':
            api_url = link['url']
            found = True
            print(api_url)
    if found == False:
        break

8
16
ServiceRequest = 6
DiagnosticReport = 8
Specimen = 2


Process the diaganostic reports

Process the reports, convert some of the objects into other datatypes.

In [365]:
import fhirclient.models.meta as meta
from dateutil import parser

def performer(my_list):
    performer = ""
    if my_list != None:
        for item in my_list:
            performer = item.display
    return performer

def performerCode(my_list):
    performer = None
    if my_list != None:
        for item in my_list:
            performer = item.identifier.value
    return performer

def codeCode(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code

    return code
def codeDisplay(concept):
    code = ""
    for coding in concept.coding:
        code = coding.display

    return code

def issued(issued):
    if issued == None:
        return None
    return parser.parse(issued.isostring)

def serviceRequest(my_list):
    sr = None
    if my_list != None:
        for item in my_list:
            if item.reference != None:
                sr = item.reference.replace('ServiceRequest/', '')
    return sr
def specimen(my_list):
    sr = None
    if my_list != None:
        for item in my_list:
            if item.reference != None:
                sr = item.reference.replace('Specimen/', '')
    return sr
def lastUpdated(meta : meta.Meta):
    if meta == None:
        return None
    return parser.parse(meta.lastUpdated.isostring)

print(len(diagnosticReports))
dfDR = pd.DataFrame([vars(s) for s in diagnosticReports])

dfDR['performerDisplay'] = dfDR['performer'].apply(performer)
dfDR['performerCode'] = dfDR['performer'].apply(performerCode)
dfDR['codingCode'] = dfDR['code'].apply(codeCode)
dfDR['codingDisplay'] = dfDR['code'].apply(codeDisplay)
dfDR['ReportLastUpdatedDate'] = dfDR['meta'].apply(lastUpdated)
dfDR['ReportIssuedDate'] = dfDR['issued'].apply(issued)
dfDR['ReportEffectiveDate'] = dfDR['effectiveDateTime'].apply(issued)
dfDR['serviceRequestId'] = dfDR['basedOn'].apply(serviceRequest)
dfDR['specimenId'] = dfDR['specimen'].apply(specimen)

dfDiagnosticReport = dfDR[['id','performerDisplay','performerCode','codingCode', 'codingDisplay', 'ReportLastUpdatedDate','ReportIssuedDate', 'ReportEffectiveDate', 'serviceRequestId','specimenId']]
dfDiagnosticReport

8


,id,performerDisplay,performerCode,codingCode,codingDisplay,ReportLastUpdatedDate,ReportIssuedDate,ReportEffectiveDate,serviceRequestId,specimenId
0,7,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-12-19 12:22:14+00:00,2025-10-14 15:59:16+00:00,NaT,None,None
1,16,NHS North West Genomics,699X0,R125.1,Thoracic aortic aneurysm or dissection (WES or...,2025-12-25 06:46:18+00:00,2025-10-14 15:59:16+00:00,2025-10-14 15:59:16+00:00,15,None
2,22,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-12-19 11:51:08+00:00,2025-10-14 15:59:16+00:00,NaT,None,None
3,61,NHS North West Genomics,699X0,R375.1,Family follow-up testing to aid variant interp...,2025-12-25 08:02:50+00:00,2025-12-17 12:22:52+00:00,2025-12-17 12:22:52+00:00,60,None
4,75,NHS North West Genomics,699X0,M80.2,"Acute Myeloid Leukaemia, Multi-target NGS pane...",2025-12-25 08:02:48+00:00,2025-12-17 12:25:36+00:00,2025-12-17 12:25:36+00:00,74,None
5,88,NHS North West Genomics,699X0,M80.21,"Acute Myeloid Leukaemia, FLT3 TKD hotspot",2025-12-28 07:04:37+00:00,2025-12-18 12:14:40+00:00,2025-12-18 12:14:40+00:00,87,156
6,99,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-28 07:03:18+00:00,2025-12-06 13:37:35+00:00,2025-12-06 13:37:35+00:00,133,143
7,119,NHS North West Genomics,699X0,R240.1,Diagnostic testing for known mutation(s) (Targ...,2025-12-25 08:02:57+00:00,2025-08-14 09:41:16+00:00,2025-08-14 09:41:16+00:00,118,None


In [366]:
print(len(specimens))
dfSP = pd.DataFrame([vars(s) for s in specimens])
dfSP['SpecimenReceivedDate'] = dfSP['receivedTime'].apply(issued)


dfSpecimen = dfSP[['id','SpecimenReceivedDate']]
dfSpecimen

2


,id,SpecimenReceivedDate
0,156,2025-12-16
1,143,2025-11-21


Clean up requests

In [367]:
print(len(serviceRequests))

def requester(item):
    performer = None
    if item != None:
        performer = item.display
    return performer

def requesterCode(item):
    performer = None
    if item != None:
        performer = item.identifier.value
    return performer

dfSR = pd.DataFrame([vars(s) for s in serviceRequests])
dfSR['requesterDisplay'] = dfSR['requester'].apply(requester)
dfSR['requesterCode'] = dfSR['requester'].apply(requesterCode)
dfSR['OrderAuthoredOnDate'] = dfSR['authoredOn'].apply(issued)

dfServiceRequest = dfSR[['id','requesterDisplay','requesterCode','OrderAuthoredOnDate']]
dfServiceRequest

6


,id,requesterDisplay,requesterCode,OrderAuthoredOnDate
0,118,None,None,2025-08-05
1,15,Nottingham University Hospitals,RX1,NaT
2,60,Manchester University NHS Foundation Trust,R0A,2025-05-04
3,74,Manchester University NHS Foundation Trust,R0A,2025-12-06
4,87,Stockport NHS Foundation Trust,RWJ,2025-12-03
5,133,Liverpool Women's NHS Foundation Trust,REP,2025-11-06


Join both dataframes into a single result.

In [368]:
# ... existing code ...
df = pd.merge(
    dfDiagnosticReport,
    dfServiceRequest,
    left_on='serviceRequestId',
    right_on='id',
    how="left",
    indicator=True,
    suffixes=('_dr', '_sr')
)
df = df.drop(columns=['_merge'])
df = pd.merge(
    df,
    dfSpecimen,
    left_on='specimenId',
    right_on='id',
    how="left",
    indicator=True,
    suffixes=('_dr', '_sp')
)


df['ReportIssuedDate'] = pd.to_datetime(df['ReportIssuedDate'], utc=True).dt.tz_localize(None)
df['OrderAuthoredOnDate'] = pd.to_datetime(df['OrderAuthoredOnDate'], utc=True).dt.tz_localize(None)
df['ReportLastUpdatedDate'] = pd.to_datetime(df['ReportLastUpdatedDate'], utc=True).dt.tz_localize(None)

df['requestedDuration'] = (df['ReportIssuedDate'] - df['OrderAuthoredOnDate']).dt.days

df['releaseDuration'] = (df['ReportLastUpdatedDate'] - df['ReportIssuedDate']).dt.days
df['testingDuration'] = (df['ReportIssuedDate'] - df['SpecimenReceivedDate']).dt.days
df['OrderToSpecimenReceivedDuration'] = (df['SpecimenReceivedDate'] - df['OrderAuthoredOnDate']).dt.days

df['releaseDuration'] = df['releaseDuration'].fillna(-1)
df['requestedDuration'] = df['requestedDuration'].fillna(-1)
df['OrderToSpecimenReceivedDuration'] = df['OrderToSpecimenReceivedDuration'].fillna(-1)

df

,id_dr,performerDisplay,performerCode,codingCode,codingDisplay,ReportLastUpdatedDate,ReportIssuedDate,ReportEffectiveDate,serviceRequestId,specimenId,...,requesterDisplay,requesterCode,OrderAuthoredOnDate,id,SpecimenReceivedDate,_merge,requestedDuration,releaseDuration,testingDuration,OrderToSpecimenReceivedDuration
0,7,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-12-19 12:22:14,2025-10-14 15:59:16,NaT,None,None,...,NaN,NaN,NaT,NaN,NaT,left_only,-1.0,65,NaN,-1.0
1,16,NHS North West Genomics,699X0,R125.1,Thoracic aortic aneurysm or dissection (WES or...,2025-12-25 06:46:18,2025-10-14 15:59:16,2025-10-14 15:59:16+00:00,15,None,...,Nottingham University Hospitals,RX1,NaT,NaN,NaT,left_only,-1.0,71,NaN,-1.0
2,22,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-12-19 11:51:08,2025-10-14 15:59:16,NaT,None,None,...,NaN,NaN,NaT,NaN,NaT,left_only,-1.0,65,NaN,-1.0
3,61,NHS North West Genomics,699X0,R375.1,Family follow-up testing to aid variant interp...,2025-12-25 08:02:50,2025-12-17 12:22:52,2025-12-17 12:22:52+00:00,60,None,...,Manchester University NHS Foundation Trust,R0A,2025-05-04,NaN,NaT,left_only,227.0,7,NaN,-1.0
4,75,NHS North West Genomics,699X0,M80.2,"Acute Myeloid Leukaemia, Multi-target NGS pane...",2025-12-25 08:02:48,2025-12-17 12:25:36,2025-12-17 12:25:36+00:00,74,None,...,Manchester University NHS Foundation Trust,R0A,2025-12-06,NaN,NaT,left_only,11.0,7,NaN,-1.0
5,88,NHS North West Genomics,699X0,M80.21,"Acute Myeloid Leukaemia, FLT3 TKD hotspot",2025-12-28 07:04:37,2025-12-18 12:14:40,2025-12-18 12:14:40+00:00,87,156,...,Stockport NHS Foundation Trust,RWJ,2025-12-03,156,2025-12-16,both,15.0,9,2.0,13.0
6,99,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-28 07:03:18,2025-12-06 13:37:35,2025-12-06 13:37:35+00:00,133,143,...,Liverpool Women's NHS Foundation Trust,REP,2025-11-06,143,2025-11-21,both,30.0,21,15.0,15.0
7,119,NHS North West Genomics,699X0,R240.1,Diagnostic testing for known mutation(s) (Targ...,2025-12-25 08:02:57,2025-08-14 09:41:16,2025-08-14 09:41:16+00:00,118,None,...,None,None,2025-08-05,NaN,NaT,left_only,9.0,132,NaN,-1.0


In [369]:
import plotly.graph_objects as go


fig = go.Figure()
dfTotals = df.requesterDisplay.value_counts().reset_index(name='counts')
fig.add_trace(go.Bar(x=dfTotals['requesterDisplay'], y=dfTotals['counts']))
fig.show()

fig2 = go.Figure()
dfTotals2 = df.codingDisplay.value_counts().reset_index(name='counts')
fig2.add_trace(go.Bar(x=dfTotals2['codingDisplay'], y=dfTotals['counts']))
fig2.show()



In [370]:
import plotly.express as px

df1 = df.groupby(['requesterCode', 'codingCode']).size().reset_index(name='count')

fig = px.bar(df1, x="requesterCode", y="count", color="codingCode")

fig.show()
#px.bar(df1, x="requesterCode", y="count", color="codingCode", barmode="group")

In [371]:
dfS = df[['SpecimenReceivedDate', 'testingDuration', 'codingCode']]
dfS = dfS.dropna(subset=['SpecimenReceivedDate'])
dfS = dfS.groupby(['SpecimenReceivedDate', 'testingDuration', 'codingCode']).size().reset_index(name='counts')
dfS


,SpecimenReceivedDate,testingDuration,codingCode,counts
0,2025-11-21,15.0,R112.1,1
1,2025-12-16,2.0,M80.21,1


In [372]:
# Now the scatter plot will work as expected
fig = px.scatter(
    dfS,
    x="SpecimenReceivedDate",
    y="testingDuration",
    color="codingCode",
    size = "counts",
    title="Testing Time from Specimen Collection to Report Release"
)

fig.show()

In [373]:
dti = pd.date_range("2025-10-01", periods=100, freq="d").to_frame(index=False, name="date")
dti['date'] = dti['date'].dt.date

df['ReportSentDT'] = df['ReportLastUpdatedDate'].dt.date
df['ReportIssuedDT'] = df['ReportIssuedDate'].dt.date
df['OrderAuthoredDT'] = df['OrderAuthoredOnDate'].dt.date
df['SpecimenReceivedDT'] = df['SpecimenReceivedDate'].dt.date

dfA = df.groupby(['ReportSentDT']).size().reset_index(name='ReportSent')
dfB = df.groupby(['ReportIssuedDT']).size().reset_index(name='ReportIssued')
dfC = df.groupby(['OrderAuthoredDT']).size().reset_index(name='OrderAuthored')
dfD = df.groupby(['SpecimenReceivedDT']).size().reset_index(name='SpecimenReceived')

df_merged = pd.merge(dti, dfA, left_on='date', right_on='ReportSentDT', how='left')
df_merged = pd.merge(df_merged, dfB, left_on='date', right_on='ReportIssuedDT', how='left')
df_merged = pd.merge(df_merged, dfC, left_on='date', right_on='OrderAuthoredDT', how='left')
df_merged = pd.merge(df_merged, dfD, left_on='date', right_on='SpecimenReceivedDT', how='left')
df_merged.drop(columns=['ReportSentDT', 'ReportIssuedDT', 'OrderAuthoredDT','SpecimenReceivedDT'], inplace=True)

dfmelt = df_merged.melt(id_vars=["date"],
        var_name="DateType",
        value_name="Count")

#dfmelt = dfmelt.groupby(['date', 'Value', 'DateType']).size().reset_index(name='counts')

fig = px.scatter(dfmelt,x="date", y="Count", color="DateType",  title="Last Updated")
fig.show()

fig = px.line(dfmelt,x="date", y="Count", color="DateType",  title="Last Updated")
fig.show()

dfmelt





,date,DateType,Count
0,2025-10-01,ReportSent,NaN
1,2025-10-02,ReportSent,NaN
2,2025-10-03,ReportSent,NaN
3,2025-10-04,ReportSent,NaN
4,2025-10-05,ReportSent,NaN
...,...,...,...
395,2026-01-04,SpecimenReceived,NaN
396,2026-01-05,SpecimenReceived,NaN
397,2026-01-06,SpecimenReceived,NaN
398,2026-01-07,SpecimenReceived,NaN


In [374]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Initialize the Dash app (no longer need JupyterDash)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.callback(
    Output('reportsByRequester', 'figure')
)
def reportsByRequester():
    fig = px.bar(dfReports, x="requesterCode", y="count", color="codingCode")
    return fig

@app.callback(
    Output('reportsByTestCode', 'figure')
)
def reportsByTestCode():
    figTC = px.bar(dfReports, x="codingCode", y="count", color="requesterCode")
    return figTC

@app.callback(
    Output('durations', 'figure')
)
def durations():
    dfS = df[['SpecimenReceivedDate', 'testingDuration', 'codingCode']]
    dfS = dfS.dropna(subset=['SpecimenReceivedDate'])
    dfS = dfS.groupby(['SpecimenReceivedDate', 'testingDuration', 'codingCode']).size().reset_index(name='counts')

    # Now the scatter plot will work as expected
    figD = px.scatter(
        dfS,
        x="SpecimenReceivedDate",
        y="testingDuration",
        color="codingCode",
        size = "counts",
        title="Testing Time from Specimen Collection to Report Release"
    )

    return figD

@app.callback(
    Output('specimens', 'figure')
)
def specimens():
    dfS = df[['OrderAuthoredOnDate', 'OrderToSpecimenReceivedDuration', 'requesterCode']]
    dfS = dfS.dropna(subset=['OrderAuthoredOnDate'])
    dfS = dfS.groupby(['OrderAuthoredOnDate', 'OrderToSpecimenReceivedDuration', 'requesterCode']).size().reset_index(name='counts')

    # Now the scatter plot will work as expected
    figD = px.scatter(
        dfS,
        x="OrderAuthoredOnDate",
        y="OrderToSpecimenReceivedDuration",
        color="requesterCode",
        size = "counts",
        title="Time from Order to Specimen Received"
    )

    return figD

@app.callback(
    Output('release', 'figure')
)
def release():
    dfS = df[['ReportIssuedDate', 'releaseDuration', 'codingCode']]
    dfS = dfS.dropna(subset=['ReportIssuedDate'])
    dfS = dfS.groupby(['ReportIssuedDate', 'releaseDuration', 'codingCode']).size().reset_index(name='counts')
    # Now the scatter plot will work as expected
    figE = px.scatter(
        dfS,
        x="ReportIssuedDate",
        y="releaseDuration",
        color="codingCode",
        size = "counts",
        title="Time from Report Release to Report Sent"
    )
    return figE

@app.callback(
    Output('lines', 'figure')
)
def lines():

    figL = px.line(dfmelt,x="date", y="Count", color="DateType",  title="Timeline overview for sent reports")
    return figL



dfReports = df.groupby(['requesterCode', 'codingCode']).size().reset_index(name='count')

fig1 = reportsByRequester()
fig3 = reportsByTestCode()
fig2 = durations()
fig4 = lines()
fig5 = specimens()
fig6 = release()

app.layout = html.Div([
    html.H1("Genomic Testing Workflow Dashboard"),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='lines', figure=fig4),
        ]),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='specimens', figure=fig5),
        ]),
        dbc.Col([
            dcc.Graph(id='durations', figure=fig2)
        ])
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='release', figure=fig6),
        ])
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='reportsByRequester', figure=fig1),
        ]),
        dbc.Col([
            dcc.Graph(id='reportByTestCode', figure=fig3),
        ])

    ])
])


if __name__ == '__main__':
    # Use app.run with jupyter_mode instead of JupyterDash's run_server
    app.run(jupyter_mode='inline', port=8051)